### Import the Librabries

In [30]:


import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests
import random


print('Libraries imported')

Libraries imported


###  Download and load Toronto Data

In [3]:
#Toronto Geographical Data on Neighbhorhoods/Boroughs
Tor_url='https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=945633050'
Tor_df = pd.read_html(Tor_url)
Tor_df_postcodes=Tor_df[0]
#print('imported dataframe has', Tor_df_postcodes['Postcode'].count(), "postcodes entries")
Tor_df_postcodes.head(10)
#df_postcodes.dtypes

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


In [20]:
Tor_df_postcodes[Tor_df_postcodes['Neighbourhood']=='Not Assigned'].count()

Postcode         0
Borough          0
Neighbourhood    0
dtype: int64

In [27]:
#Preprocessing
#Toronto - Get rid of the "Not assigned" / Replace

Tor_df_postcodes = Tor_df_postcodes[Tor_df_postcodes["Borough"] != "Not assigned"]

#Replace the N/A values of Neighborhood with the Borough Value
#Tor_df_postcodes["Neighbourhood"].replace("Not assigned", Tor_df_postcodes["Borough"], inplace=True)

Tor_df_postcodes.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [28]:
#Now combine all the neighbourhoods with the same postal codes
Tor_df = Tor_df_postcodes.groupby(["Postcode","Borough"])["Neighbourhood"].apply(list)
Tor_df = Tor_df.sample(frac=1).reset_index()
Tor_df["Neighbourhood"] = Tor_df["Neighbourhood"].str.join(', ')
Tor_df.head(10)

,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M1W,Scarborough,L'Amoreaux West
2,M1B,Scarborough,"Rouge, Malvern"
3,M2P,North York,York Mills West
4,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
5,M3M,North York,Downsview Central
6,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade
7,M2R,North York,Willowdale West
8,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo..."
9,M4G,East York,Leaside


In [29]:
Tor_df.shape

(103, 3)

In [31]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [32]:
#Get the Latitude and Langitude to match the Postcode
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
geo = pd.read_csv("Geospatial_Coordinates.csv", index_col = 0)         
Tor_df = Tor_df.join(geo, on = "Postcode")

#Take only those neighborhoods that contain the word Toronto
Tor_df = Tor_df[Tor_df['Borough'].str.contains('Toronto')].reset_index(drop=True)
Tor_df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5W,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
1,M5V,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
2,M4S,Central Toronto,Davisville,43.704324,-79.388790
3,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,M4P,Central Toronto,Davisville North,43.712751,-79.390197


In [33]:
Toraddress = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
Tlocation = geolocator.geocode(Toraddress)
Tlatitude = Tlocation.latitude
Tlongitude = Tlocation.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(Tlatitude,Tlongitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [34]:
Tor_df.rename(columns = {"Neighbourhood":"Neighborhood"}, inplace = True)

In [36]:
Tor_df_copy=Tor_df.copy()
Tor_df_copy.drop("Postcode",axis = 1,inplace=True)


In [37]:
Tor_df_copy.head()

,Borough,Neighborhood,Latitude,Longitude
0,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,43.646435,-79.374846
1,Downtown Toronto,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420
2,Central Toronto,Davisville,43.704324,-79.388790
3,Downtown Toronto,Berczy Park,43.644771,-79.373306
4,Central Toronto,Davisville North,43.712751,-79.390197


In [38]:
map_toronto = folium.Map(location=[Tlatitude, Tlongitude], zoom_start=11, width='100%', height='100%')

for lat, lng, borough, neighborhood in zip(Tor_df_copy['Latitude'], Tor_df_copy['Longitude'], Tor_df_copy['Borough'], Tor_df_copy['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)

map_toronto

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(Tor_df_copy[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = Tor_df_copy['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

(39, 40)

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,"Adelaide, King, Richmond",Berczy Park,"Brockton, Exhibition Place, Parkdale Village",Business Reply Mail Processing Centre 969 Eastern,"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara","Cabbagetown, St. James Town",Central Bay Street,"Chinatown, Grange Park, Kensington Market",Christie,...,Roselawn,"Runnymede, Swansea","Ryerson, Garden District",St. James Town,Stn A PO Boxes 25 The Esplanade,Studio District,"The Annex, North Midtown, Yorkville",The Beaches,"The Beaches West, India Bazaar","The Danforth West, Riverdale"
0,"Adelaide, King, Richmond",1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Berczy Park,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Brockton, Exhibition Place, Parkdale Village",0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Business Reply Mail Processing Centre 969 Eastern,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)